In [ ]:
def wrapper(d_lr = 2e-4, g_lr = 2e-4):
    import tensorflow as tf
    import tensorflow.contrib.slim as slim
    import numpy as np
    import hops.hdfs as hdfs
    from hops import tensorboard
    import sys
    import time
    import os
    
    project_DIR = hdfs.project_path()
    faces = "hdfs:///Projects/ID2223_Project_Gans_new/faces/celeb_64x64.npy"
    catizh = "hdfs:///Projects/ID2223_Project_Gans_new/Cats/Cats_64x64.npy"
    
    with hdfs.get_fs().open_file(catizh, "r") as someFile:
        importNumpyCats = np.load(someFile)
    
    CatImages = np.reshape(importNumpyCats, (-1,64,64,3))
    
    org_CatImages = CatImages
    logdir = tensorboard.logdir()

    if not os.path.exists(logdir + '/train'):
        os.mkdir(logdir + '/train')
    
    image = CatImages[0]
    IMAGE_XDIM = image.shape[0]
    IMAGE_YDIM = image.shape[1]
    IMAGE_ZDIM = image.shape[2]
    
    shape = len(CatImages[0]), IMAGE_XDIM, IMAGE_YDIM, IMAGE_ZDIM
    out_channel_dim = shape[3]
    
    def normalizeImages(x):
        # normalise to -1 to 1, tanH
        v_min = x.min(axis=(1, 2), keepdims=True)
        v_max = x.max(axis=(1, 2), keepdims=True)
        return 2. * (x - v_min)/(v_max - v_min) - 1.
    
    def reScale(x):
        return (x + 1)/2
    
    def lrelu(inputs, leak=0.2, scope="lrelu"):
        with tf.variable_scope(scope):
            f1 = 0.5 * (1 + leak)
            f2 = 0.5 * (1 - leak)
            return f1 * inputs + f2 * abs(inputs)
        
    def convLayer(inputs, filters, kernel_size, strides, padding, activation, batch_normalization, training):
        conv = tf.layers.conv2d(inputs=inputs, 
                               filters=filters, 
                               kernel_size=kernel_size, 
                               strides=strides, 
                               padding=padding)
        if activation == "relu" and batch_normalization == True:
            return tf.nn.relu(tf.layers.batch_normalization(conv, training=training))
        elif activation =="relu" and batch_normalization == False:
            return tf.nn.relu(conv)
        elif activation =="lrelu" and batch_normalization == True:
            return lrelu(tf.layers.batch_normalization(conv, training=training), 0.2)
        elif activation =="lrelu" and batch_normalization == False:
            return lrelu(conv, 0.2)
        else:
            return conv
    
    def deConvLayer(inputs, filters, kernel_size, strides, padding, activation, batch_normalization, training):
        deConv = tf.layers.conv2d_transpose(inputs=inputs, 
                                       filters=filters, 
                                       kernel_size=kernel_size, 
                                       strides=strides, 
                                       padding=padding)
        if activation == "relu" and batch_normalization == True:
            return tf.nn.relu(tf.layers.batch_normalization(deConv, training=training))
        elif activation =="relu" and batch_normalization == False:
            return tf.nn.relu(deConv)
        elif activation =="lrelu" and batch_normalization == True:
            return lrelu(tf.layers.batch_normalization(deConv, training=training), 0.2)
        elif activation =="lrelu" and batch_normalization == False:
            return lrelu(deConv,0.2)
        else:
            return deConv
        
    def generator(x, out_channel_dim, isTrain=True, reuse=False):
        with tf.variable_scope('G', reuse=reuse):
            
            h1 = deConvLayer(inputs=x, filters=1024, kernel_size=[4, 4], strides=(1,1), padding='VALID', 
                           activation = 'lrelu', batch_normalization = True, training=isTrain)
            
            h2 = deConvLayer(inputs=h1, filters=512, kernel_size=[4, 4], strides=(2,2), padding='SAME', 
                           activation = 'lrelu', batch_normalization = True, training=isTrain)
            
            h3 = deConvLayer(inputs=h2, filters=256, kernel_size=[4, 4], strides=(2,2), padding='SAME', 
                           activation = 'lrelu', batch_normalization = True, training=isTrain)
            
            h4 = deConvLayer(inputs=h3, filters=128, kernel_size=[4, 4], strides=(2,2), padding='SAME', 
                           activation = 'lrelu', batch_normalization = True, training=isTrain)
            # out layer
            h5 = tf.layers.conv2d_transpose(inputs= h4, filters=out_channel_dim, 
                                            kernel_size=[4, 4], strides=(2, 2), 
                                            padding='SAME')
            o = tf.nn.tanh(h5)
            tf.summary.image("Generated Images", o , 9)
            return o

    def discriminator(x, isTrain=True, reuse=False):
        with tf.variable_scope('D', reuse=reuse):
            
            h1 = convLayer(inputs=x, filters=128, kernel_size=[4, 4], strides=(2, 2), padding='SAME', 
                           activation='lrelu', batch_normalization=True, training=isTrain)
            
            h2 = convLayer(inputs=h1, filters=256, kernel_size=[4, 4], strides=(2, 2), padding='SAME', 
                           activation='lrelu', batch_normalization=True, training=isTrain)
            
            h3 = convLayer(inputs=h2, filters=512, kernel_size=[4, 4], strides=(2, 2), padding='SAME', 
                           activation='lrelu', batch_normalization=True, training=isTrain)
            
            h4 = convLayer(inputs=h3, filters=1024, kernel_size=[4, 4], strides=(2, 2), padding='SAME', 
                           activation='lrelu', batch_normalization=True, training=isTrain)
            # out Layer
            h5 = tf.layers.conv2d(inputs=h4, filters=1, kernel_size=[4, 4], strides=(1, 1), padding='VALID')
            o = tf.nn.sigmoid(h5)
            return o, h5

    def getNextBatch(x, batch_size):
        nImg = x.shape[0]
        idx = np.random.randint(0, nImg,size = batch_size)
        return x[idx,:]
    
    tf.reset_default_graph()
    
    # Defining Placeholder
    input_real_images = tf.placeholder(shape=[None, 64, 64, 3], dtype=tf.float32, name='input_real_images') 
    input_z = tf.placeholder(dtype=tf.float32, shape=[None,1,1, 100], name='input_z')
    learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
    X_viz = tf.placeholder(tf.float32, shape=[3, 64, 64, 3]) # input Image
    isTrain = tf.placeholder(dtype=tf.bool)

    label_smoothing = 0.9
    
    # Defining the Networks
    g_model = generator(input_z, out_channel_dim, isTrain)
    d_model_real, d_logits_real = discriminator(input_real_images, isTrain)
    d_model_fake, d_logits_fake = discriminator(g_model,isTrain, reuse=True)

    # Defining Loss
        # Discriminator Loss
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_model_real) * label_smoothing ))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_model_fake)))
    d_loss = d_loss_real + d_loss_fake
        # Generator Loss                                           
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_model_fake) * label_smoothing))
    
    # Gathering Variables
    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if var.name.startswith('D')]
    g_vars = [var for var in t_vars if var.name.startswith('G')]

    # Defining Optimizer
    d_beta1 = 0.5; g_beta1 = 0.5
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): 
        d_train_opt = tf.train.AdamOptimizer(d_lr, beta1=d_beta1).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(g_lr, beta1=g_beta1).minimize(g_loss, var_list=g_vars)
    
    # Training Parameters
    steps = 1 
    batch_size = 64
    z_dim = 100
    epochs = 1000
    iteration = int(CatImages.shape[0] / batch_size)
    save_step = 10
    # Normalise to -1.0 to 1.0, tanH
    CatImages = normalizeImages(org_CatImages)
    # Start Training Session
    with tf.Session() as sess:
        # Values to be added to tensorboard
        tf.summary.scalar('Generator_loss', g_loss)
        tf.summary.scalar('Discriminator_loss', d_loss)
        tf.summary.image("Input_images", X_viz)
        merged = tf.summary.merge_all()
        # Define writer
        writer_train = tf.summary.FileWriter( logdir + "/train", sess.graph )
        sess.run(tf.global_variables_initializer())
        for epoch_i in range(epochs):
            for i in range(iteration):
                batch_images = getNextBatch(CatImages, batch_size) # MiniBatch real img
                batch_z = np.random.uniform(-1.0, 1.0, size=(batch_size,1,1, z_dim)) # MiniBatch noise
                _ = sess.run(d_train_opt, feed_dict={input_real_images: batch_images, input_z: batch_z, isTrain:True}) # train Discriminator
                _ = sess.run(g_train_opt, feed_dict={input_real_images: batch_images, input_z: batch_z, isTrain:True}) # train Generator
                if steps == 1 or steps % save_step == 0: # record losses and retrevie images every ...save_step
                    z_batch = np.random.uniform(-1.0, 1.0, size=[batch_size,1,1, z_dim]) # MiniBatch noise
                    x_vizu_3 = reScale(getNextBatch(batch_images, 3)) # Example training images from current batch
                    summary_train = sess.run(merged, feed_dict = {X_viz: x_vizu_3, input_real_images: batch_images, input_z: z_batch,isTrain:False})
                    writer_train.add_summary(summary_train, steps)
                steps += 1





Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7532,application_1513605045578_5079,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [ ]:
from hops import tflauncher
tensorboard_hdfs_logdir = tflauncher.launch(spark, wrapper)

In [ ]:
from hops import tensorboard
# Visualize all TensorBoard events for the jobs in the same TensorBoard
tensorboard.visualize(spark, tensorboard_hdfs_logdir)